In [22]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import os
import uuid

In [23]:

etl = 'data_request'

raw_data_request_dir = ''
final_data_request_dir = ''

if etl == 'data_request':
    raw_data_request_dir   = 'data/data_request/raw_data.v1.07.11.2019/'
    final_data_request_dir = 'data/data_request/request.v1.07.11.2019/'

## RESIDENT DF

In [24]:
_resident_df = pd.read_csv(raw_data_request_dir + 'resident' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_resident_df = _resident_df.replace(np.nan,'',regex=True)

_nabitasan = ['jepril@gmail.com','mark@gmail.com']

_organization = _nabitasan

_resident_df = _resident_df[_resident_df['user-cam.id'].isin(_organization) ]
_resident_df = _resident_df.drop(['profile_picture.name',
                                  'address.country',
                                  'type',
                                  'id',
                                  'user-cam.id',
                                  'contact_number.country_code',
                                  'last_name_suffix',
                                  'profile_picture.path',
                                  'identification.id2.identifier',
                                  'identification.id2.type',
                                  'identification.id3.identifier',
                                  'identification.id3.type'],axis=1)
_resident_df.head(3)

,_id,_index_map,registered_at,birthdate,email_address,organization,first_name,nhid,gender,middle_name,last_name,user-cam.owner,address.postal_code,address.barangay,address.province,contact_number.number,identification.id1.identifier,identification.id1.type,address.lot_or_house_number
0,0005038a-b9b9-4fe6-ac06-2ce39d6da621,,10/29/2018 at 04:18:34 PM GMT+08:00,03/09/1980,,Pototan RHU,Rodolfo,,Male,Magno,Pasigay,mark@gmail.com,5008,nabitasan,pototan,,,4Ps,
28,0147add0-167b-4047-ae4e-dec044dcf9ad,,10/26/2018 at 03:33:55 PM GMT+08:00,12/13/1990,,Pototan RHU,Analiza,,Female,Gambito,Molate,mark@gmail.com,,nabitasan pototan,iloilo,,,4Ps,
36,016db072-e5e0-4ac0-92d1-3c43677867b0,,11/27/2018 at 12:00:50 PM GMT+08:00,03/30/2017,,Pototan RHU,Remah Elle,,Female,Ligno,Virzo,mark@gmail.com,5008,nabitasan,iloilo,,,4Ps,


In [25]:
list(_resident_df.columns)

['_id',
 '_index_map',
 'registered_at',
 'birthdate',
 'email_address',
 'organization',
 'first_name',
 'nhid',
 'gender',
 'middle_name',
 'last_name',
 'user-cam.owner',
 'address.postal_code',
 'address.barangay',
 'address.province',
 'contact_number.number',
 'identification.id1.identifier',
 'identification.id1.type',
 'address.lot_or_house_number']

## PROFILE DF

In [26]:
_profile_df = pd.read_csv(raw_data_request_dir + 'profiles' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_profile_df = _profile_df.replace(np.nan,'',regex=True)[['_id','_index_map','profiles.civil_status']].reset_index(drop=True)

_profile_df = _profile_df[_profile_df['_index_map'] == '0']
_profile_df = _profile_df.drop('_index_map',axis=1)
_profile_df.head(4)


,_id,profiles.civil_status
0,0005038a-b9b9-4fe6-ac06-2ce39d6da621,Married
1,0014f56d-c217-4fe9-afa4-291a0c0a23d2,Single
2,001522fc-183a-468e-a314-dbe99f5dd008,Married
5,001e0790-bd7c-410f-8950-8497bffa4338,Married


## HEALTH INFORMATION DF

In [27]:
_hi_df = pd.read_csv(raw_data_request_dir + 'health_informations' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_hi_df = _hi_df.replace(np.nan,'',regex=True).reset_index(drop=True)


_hi_df = _hi_df[_hi_df['_index_map'] == '0']
_hi_df = _hi_df.drop('_index_map',axis=1)
#_profile_df.head(4)
_hi_df.head(3)

,_id,health_informations.date_updated,health_informations.waist_circumference,health_informations.weight,health_informations.exercise_in_a_week,health_informations.blood_type,health_informations.blood_sign,health_informations.fruits_in_a_week,health_informations.allergies,health_informations.smoking_habit,...,health_informations.vegetables_in_a_week,health_informations.high_cost_medicine,health_informations.maintenance_drugs,health_informations.height,health_informations.blood_pressure.first_reading.systole,health_informations.blood_pressure.third_reading.diastole,health_informations.blood_pressure.third_reading.systole,health_informations.blood_pressure.second_reading.systole,health_informations.blood_pressure.first_reading.diastole,health_informations.blood_pressure.second_reading.diastole
0,0005038a-b9b9-4fe6-ac06-2ce39d6da621,11/12/2018 at 07:33:04 PM GMT+08:00,,60,More than 5 hours,I don't know,I don't know,2,,Active,...,2,No,No,161,,,,,,
1,0014f56d-c217-4fe9-afa4-291a0c0a23d2,04/20/2018 at 09:50:02 AM GMT+08:00,,55,0 to 1 hour,B,+,1,none,Never,...,2,No,No,5.7,,,,,,
3,001522fc-183a-468e-a314-dbe99f5dd008,05/21/2010 at 01:49:49 AM GMT+08:00,,38,0 to 1 hour,I don't know,I don't know,1,,Never,...,1,No,No,5,120,,,120,80,90


## HEALTH DIAGNOSIS

In [28]:
_diagnosis_df = pd.read_csv(raw_data_request_dir + 'health_informations.diagnosed' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_diagnosis_df = _diagnosis_df.replace(np.nan,'',regex=True).reset_index(drop=True)


#_hi_df = _hi_df[_hi_df['_index_map'] == '0']
_diagnosis_df = _diagnosis_df.drop('_index_map',axis=1)
_diagnosis_df = _diagnosis_df.groupby('_id').agg(lambda x: x.tolist())
_diagnosis_df = _diagnosis_df.reset_index()
_diagnosis_df.head(13)

,_id,health_informations.diagnosed
0,014d6a09-171b-45bb-9c86-82e3cad2be2e,[]
1,017669ed-6cf5-4b7d-a858-a589a862afb0,"[Hypertension, Hypertension, Hypertension]"
2,019758f2-14fb-46d8-8c25-b25a178e4389,[Hypertension]
3,02729bf0-8440-4323-bc18-8b0e4b7199f0,[Asthma]
4,0339fda8-83f7-4f50-bb38-82e148bc2d33,[Heart Failure]
5,047b2190-9ea5-4364-9e5c-fadadf22bfa4,[Diabetes]
6,04c54209-620d-4210-96fa-619b31f5955c,"[Hypertension, Hypertension]"
7,05911e9c-6a64-437e-8919-b1e4588bd8d3,[]
8,05c91ab4-515c-40d9-ba6f-8134d0a55661,[Diabetes]
9,05f6cc29-23b5-49be-b3d6-72f48668c854,[Diabetes]


In [31]:
_merge_df = _resident_df.merge(_profile_df,on=["_id"], how="left",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_hi_df,on=["_id"], how="left",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_diagnosis_df,on=["_id"], how="left",  suffixes=('_x', '_y') )
_merge_df.T

,0,1,2,3,4,5,6,7,8,9,...,477,478,479,480,481,482,483,484,485,486
_id,0005038a-b9b9-4fe6-ac06-2ce39d6da621,0147add0-167b-4047-ae4e-dec044dcf9ad,016db072-e5e0-4ac0-92d1-3c43677867b0,0214972f-7638-4465-a63b-0d15cbeb5eb7,02615ac0-2593-43ba-a55c-00d535e21b83,036bc91b-91bf-4593-be7a-781046c4859f,03c0b97e-0f15-4879-a091-a2c2505b4c2c,03d8440d-c267-44fe-93ca-8f82de193648,045881a4-6ef6-400d-b51f-eb2ba579043e,047eef21-0259-4bc3-89fe-cfc082162c01,...,f862a614-630f-4115-b775-6f288ccf53b8,f8bf57c8-82f8-4147-8b45-cacdd5fe12f3,fa107e6e-118c-43ab-a6d8-d4f34f954a94,fa2b412d-f68d-46ce-a30e-f480b16df87c,fa6353fe-6fc4-49d1-8392-968b2280a9f9,facfaea8-c74d-4688-aca7-b1b9c10200d4,fb90e010-6756-4d93-9d93-8dd169e5686f,fc1c4152-1072-43fc-8fb8-e42903582764,fc756ffd-0d9b-4b32-8eb7-4e5b82cb7ad0,ff6617b8-a9f3-449d-ae98-3b0f1a8ba260
_index_map,,,,,,,,,,,...,,,,,,,,,,
registered_at,10/29/2018 at 04:18:34 PM GMT+08:00,10/26/2018 at 03:33:55 PM GMT+08:00,11/27/2018 at 12:00:50 PM GMT+08:00,10/19/2018 at 02:57:05 PM GMT+08:00,10/14/2018 at 06:45:09 AM GMT+08:00,10/19/2018 at 03:16:06 PM GMT+08:00,10/20/2017 at 09:09:15 AM GMT+08:00,02/19/2019 at 06:57:07 PM GMT+08:00,02/19/2019 at 06:16:36 PM GMT+08:00,11/09/2018 at 05:02:08 PM GMT+08:00,...,11/08/2018 at 09:03:26 AM GMT+08:00,10/18/2018 at 11:12:30 AM GMT+08:00,10/16/2018 at 04:50:35 PM GMT+08:00,10/22/2018 at 11:37:40 AM GMT+08:00,10/21/2018 at 07:15:36 AM GMT+08:00,11/27/2018 at 10:33:54 AM GMT+08:00,02/19/2019 at 05:08:40 PM GMT+08:00,02/19/2019 at 06:29:06 PM GMT+08:00,10/26/2018 at 03:28:07 PM GMT+08:00,10/22/2018 at 02:24:43 PM GMT+08:00
birthdate,03/09/1980,12/13/1990,03/30/2017,04/13/1988,11/28/1987,01/26/2012,07/13/2001,03/26/1980,01/15/2001,12/24/2017,...,07/10/1989,12/08/2017,10/15/1983,01/18/2017,05/17/2003,08/07/1981,05/08/1999,11/21/1984,05/20/2016,06/21/2016
email_address,,,,,,,,,,,...,,,,,,,,,,
organization,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,...,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU,Pototan RHU
first_name,Rodolfo,Analiza,Remah Elle,Charlie,Mary Grace,Princess Angel,Karen Kate,Anthony,Johnrey,Brylle Arren,...,Linaly,Mary Jane,Gerald,John Lester,Mae,Evangeline,Rachelle,Ruby,Clark Omar,Arcyah Huxen Vianey
nhid,,,,,,,,,,,...,,,,,,,,,,
gender,Male,Female,Female,Male,Female,Female,Female,Male,Male,Male,...,Female,Female,Male,Male,Female,Female,Female,Female,Male,Female
middle_name,Magno,Gambito,Ligno,Luca,Majarucon,Toledo,Perra,Telesforo,Ligno,Porras,...,Porras,Porras,Alcantara,Toledo,Laquiores,Victoriano,Catotal,Porcal,Perra,Custodio


## WRITE TO CSV

In [32]:
_merge_df.to_csv(final_data_request_dir + 'final' + '.csv',index=False)